## Урок 2

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
from gensim.models import LdaModel
from gensim.test.utils import datapath

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import re
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.4f}'.format)

In [ ]:
# !pip install gensim библиотека для «Тематического моделирования»
# !pip install razdel- токенизация
# !pip install pymorphy2 - приведение слов в нормальную форму
# !pip install nltk - токенизация лемматизация и т.д.

In [6]:
news = pd.read_csv(r"./articles.csv")
news.head()

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...
3,4898,Главный тренер «Кубани» Юрий Красножан прокомм...
4,4899,Решением попечительского совета владивостокско...


In [7]:
users = pd.read_csv(r"./users_articles.csv")
users.head()

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"
3,u101138,"[5933, 6186, 5055, 6977, 5206, 488389]"
4,u108248,"[707, 1144, 2532, 2928, 3133, 324592]"


In [9]:
import nltk
nltk.download('stopwords')
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anokiro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


151


In [10]:
with open(r"./stopwords.txt") as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [11]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [12]:
%%time
#Запускаем лемматизацию текста.
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 5min 29s


In [18]:
#сформируем список текстов
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [19]:
%%time
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 39.1 s


In [20]:
# Сохраним модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [21]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(5, 0.20595577), (6, 0.35503557), (9, 0.41919035)]

In [22]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: вода северный полицейский сотрудник сша еда ночью
topic_1: научный поверхность общество администрация выборы вице-премьер кандидат
topic_2: памятник перевод остаток свердловский лесной снежный гостиница
topic_3: фонд n— технология федеральный депутат это государственный
topic_4: газ com резерв методика армения приземлиться мастер-класс
topic_5: гражданин проверка мозг налог который год закон
topic_6: год погибнуть километр который город стать nn
topic_7: это который путин рост новый россия мочь
topic_8: человек врач квартира убийство болезнь это страдать
topic_9: это год который мочь всё весь время
topic_10: год который это человек территория данные также
topic_11: эксперимент топливо завод конкурс треть сообщество характерный
topic_12: банк nnn nn век который белый год
topic_13: ребёнок родитель турция детский семья террорист бесплатный
topic_14: станция взрыв рост год космос китай китайский
topic_15: компьютерный бензин компьютер антонов палатка посадочный соотношение
topic_

In [23]:
# функция, которая будет возвращать векторное представление новости
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [24]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.1125,0.0000,0.0000,0.0000,0.0000,0.1272,0.0000,0.0000
1,4896,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3228,0.0000,0.0000,...,0.0000,0.0000,0.2514,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,4897,0.0000,0.0000,0.0000,0.0000,0.0000,0.2058,0.3551,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,4898,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0907,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,4899,0.0000,0.0000,0.0000,0.0000,0.0000,0.8741,0.0996,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [25]:
# векторные представления пользователей
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03086006, 0.14281985, 0.13360804, 0.05409457, 0.17208864,
       0.25342208, 0.        , 0.        , 0.02755622, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.17538275, 0.        , 0.        , 0.        ])

Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке) Повторить п.2, но используя уже не медиану, а max

Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [26]:
def get_user_embedding(user_articles_list, agg_type='mean'):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if agg_type == 'mean':
        user_vector = np.mean(user_vector, 0)
    if agg_type == 'median':
        user_vector = np.median(user_vector, 0)
    if agg_type == 'max':
        user_vector = np.max(user_vector, 0)
    return user_vector

In [28]:
agg_list = ['mean', 'median', 'max']
result_dict = {}
result_dict['metrics'] = ['thresholds', 'fscore', 'precision', 'recall', 'roc_auc_score']
for agg_type in agg_list:
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
        lambda x: get_user_embedding(x, agg_type=agg_type), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] +
                                      ['topic_{}'.format(i) for i in range(25)]]
    
    target = pd.read_csv(r"./users_churn.csv")
    X = pd.merge(user_embeddings, target, 'left')
    
    # разделение на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], random_state=0)
    # обучение
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    # предсказания
    y_pred = logreg.predict(X_test)
    y_proba = logreg.predict_proba(X_test)[:, 1]
    # метрики качества
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    result_dict[agg_type] = [thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score(y_test, y_proba)]
df_result = pd.DataFrame(result_dict)
df_result

,metrics,mean,median,max
0,thresholds,0.3018,0.3098,0.3097
1,fscore,0.7321,0.7638,0.7660
2,precision,0.7336,0.7376,0.7123
3,recall,0.7306,0.7918,0.8286
4,roc_auc_score,0.9616,0.9716,0.9616


Использование mean в качестве усреднения занижает важность темы для пользователей(разносторонних), которые читают новости. Использование max в качестве усреднения более полно характеризует интересы пользователя, а медиана дает промежуточный вариант.